<a href="https://colab.research.google.com/github/manu156/PGUI/blob/main/Q1%2C2%2C3%2C4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tldextract

In [ ]:
import json
from googlesearch import search
import tldextract
import requests
from bs4 import BeautifulSoup
import csv
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import nltk
import string
import math
import bisect
import os
import pandas as pd
import timeit

try:
    nltk.data.find('tokenizers/stopwords')
except LookupError:
    nltk.download('stopwords')
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Question 1

In [ ]:
query = ['Computer Science', 'IITs in India', 'Cities of Chattisgarh']
dfd = []
if not os.path.exists('docs'):
    os.makedirs('docs')
# store all documents in 'docs' directory
for q in range(len(query)):
    i = 0
    for url in search(query[q], tld='co.in', num=30,start= 1, stop=30, pause=2):
        headers = requests.utils.default_headers()
        headers.update({
            'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
        })
        try:
            resp = requests.get(url, headers=headers, timeout=10)
        except requests.exceptions.Timeout:
            continue
        soup = BeautifulSoup(resp.content, 'html5lib')
        paras = '\n'.join(map(lambda x: x.get_text(), soup.find_all('p')[:4]))
        dfd.append({'file_name': 'd'+str(i+1)+'_'+'q'+str(q+1), 'document_id': q*20+i, 'webpage_link': url, 'content': paras})
        with open(os.path.join('docs', 'd'+str(i+1)+'_'+'q'+str(q+1)), 'w') as f:
            f.write(paras)
        i += 1
        if i == 20: # break out of the loop when we get 20 docs
            break
# write details to file: name of file, IDs, webpage URLs, and content
df = pd.DataFrame(dfd)
df.to_csv('details.csv')

# inverted index: inv_index
# Documnet ID numbering is started from 0 (0-based)
inv_index = {}
stop_words = set(stopwords.words('english')) | {' '}
ps = PorterStemmer()
for p in range(3):
    for q in range(20):
        with open(os.path.join('docs', 'd' + str(q + 1) + '_' + 'q' + str(p + 1)), 'r') as f:
            # split words, strip spaces, change to lower-case, remove punctuation marks, and remove stop-words
            cdoc = [w.strip().lower().translate(str.maketrans('', '', string.punctuation)) for s in f.readlines()
                    for w in s.split(" ") if
                    w.strip().lower().translate(str.maketrans('', '', string.punctuation)) not in stop_words]
            # stemming (and remove empty strings)
            cdoc = [ps.stem(w) for w in cdoc if w != '']
            for i in cdoc:
                if i in inv_index:
                    if inv_index[i][-1] != p * 20 + q:
                        inv_index[i].append(p * 20 + q)
                else:
                    inv_index[i] = [p * 20 + q]

# conunt (three) most and (three) least used terms
mc, mw = [-1] * 3, [None] * 3
lc, lw = [math.inf] * 3, [None] * 3
for k, v in inv_index.items():
    if len(v) > min(mc):
        mw.remove(mw[mc.index(min(mc))])
        mw.append(k)
        mc.remove(min(mc))
        mc.append(len(v))
    if len(v) < max(lc):
        lw.remove(lw[lc.index(max(lc))])
        lw.append(k)
        lc.remove(max(lc))
        lc.append(len(v))

print('Postings for three most used terms:')
for i in range(3):
    print('\tTerm: ' + mw[i] + '\n\tList: ' + str(inv_index[mw[i]]))
print('Postings for three least used terms:')
for i in range(3):
    print('\tTerm: ' + lw[i] + '\n\tList: ' + str(inv_index[lw[i]]))
N = len(dfd)

Postings for three most used terms:
	Term: comput
	List: [0, 1, 2, 4, 6, 7, 10, 11, 12, 14, 15, 16, 17, 18, 19, 34, 37]
	Term: scienc
	List: [0, 1, 2, 4, 6, 7, 10, 11, 14, 15, 16, 17, 18, 21, 31, 34, 36, 37]
	Term: india
	List: [20, 23, 30, 31, 33, 34, 36, 37, 41, 43, 45, 48, 51, 53, 54, 57]
Postings for three least used terms:
	Term: disciplin
	List: [0]
	Term: draw
	List: [0]
	Term: therefor
	List: [0]


# Question 2

In [ ]:
def intersectp(a1, a2):
    """
    Merge algorithm
    """
    b = []
    i, j = 0, 0
    while i < len(a1) and j < len(a2):
        if a1[i] == a2[j]:
            b.append(a1[i])
            i += 1
            j += 1
        elif a1[i] < a2[j]:
            i += 1
        else:
            j += 1
    return b


def boolq(ql, inv_index):
    """
    Boolean query function using intersectp (merge algorithm)
    """
    ps = PorterStemmer()
    query_list = [ps.stem(w.strip().lower()) for w in ql]
    for i in query_list:
        if i not in inv_index:
            return []
    lens = [(len(inv_index[query_list[i]]), inv_index[query_list[i]]) for i in range(len(query_list))]
    lens.sort(key=lambda x: x[0])
    while len(lens) > 1:
        c = intersectp(lens.pop()[1], lens.pop()[1])
        bisect.insort(lens, (len(c), c))
    return lens[0][1]


query = ['Computer Science', 'IITs in India', 'Cities of Chattisgarh']
dfd = []
if not os.path.exists('docs'):
    os.makedirs('docs')
for q in range(len(query)):
    i = 0
    for url in search(query[q], tld='co.in', num=30, start=1, stop=30, pause=2):
        headers = requests.utils.default_headers()
        headers.update({
            'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
        })
        try:
            resp = requests.get(url, headers=headers, timeout=10)
        except requests.exceptions.Timeout:
            continue
        soup = BeautifulSoup(resp.content, 'html5lib')
        paras = '\n'.join(map(lambda x: x.get_text(), soup.find_all('p')[:4]))
        dfd.append(
            {'file_name': 'd' + str(i + 1) + '_' + 'q' + str(q + 1), 'document_id': q * 20 + i, 'webpage_link': url,
                'content': paras})
        with open(os.path.join('docs', 'd' + str(i + 1) + '_' + 'q' + str(q + 1)), 'w') as f:
            f.write(paras)
        i += 1
        if i == 20:
            break
# write details to file: name of file, IDs, webpage URLs, and content
df = pd.DataFrame(dfd)
df.to_csv('details.csv')

# inverted index: inv_index
# Documnet ID numbering is started from 0 (0-based)
inv_index = {}
stop_words = set(stopwords.words('english')) | {' '}
ps = PorterStemmer()
for p in range(3):
    for q in range(20):
        with open(os.path.join('docs', 'd' + str(q + 1) + '_' + 'q' + str(p + 1)), 'r') as f:
            # split words, strip spaces, change to lower-case, remove punctuation marks, and remove stop-words
            cdoc = [w.strip().lower().translate(str.maketrans('', '', string.punctuation)) for s in f.readlines()
                    for w in s.split(" ") if
                    w.strip().lower().translate(str.maketrans('', '', string.punctuation)) not in stop_words]
            # stemming (and remove empty strings)
            cdoc = [ps.stem(w) for w in cdoc if w != '']
            for i in cdoc:
                if i in inv_index:
                    if inv_index[i][-1] != p * 20 + q:
                        inv_index[i].append(p * 20 + q)
                else:
                    inv_index[i] = [p * 20 + q]

# query: IIT AND Computer
print('query: IIT AND Computer; ' + 'list of document IDs matching: ', boolq(['IIT', 'Computer'], inv_index))
# output for our run: [33, 38]

# Computer AND Bhilai
print('query: Computer AND Bhilai; ' + 'list of document IDs matching: ', boolq(['Computer', 'Bhilai'], inv_index))
# output for our run: []

# Bhilai AND Computer AND IIT
print('query: Bhilai AND Computer AND IIT; ' + 'list of document IDs matching: ', boolq(['Bhilai', 'Computer', 'IIT'], inv_index))
# output for our run: []


query: IIT AND Computer; list of document IDs matching:  [34, 37]
query: Computer AND Bhilai; list of document IDs matching:  []
query: Bhilai AND Computer AND IIT; list of document IDs matching:  []


In [ ]:
print(timeit.timeit("boolq(['IIT', 'Computer'], inv_index)", 'from __main__ import boolq, inv_index', number=100))
print(timeit.timeit("boolq(['Computer', 'Bhilai'], inv_index)", 'from __main__ import boolq, inv_index', number=100))
print(timeit.timeit("boolq(['Bhilai', 'Computer', 'IIT'], inv_index)", 'from __main__ import boolq, inv_index', number=100))

0.009093049000057363
0.005048134999924514
0.007286062000048332


#Question 3

In [ ]:
class Node:
    def __init__(self, key):
        self.key = key
        self.next = None

    def set_key(self, data):
        self.data = data

    def set_next(self, arg):
        self.next = arg


class SkipNode(Node):
    def __init__(self, key):
        super().__init__(key)
        self.skip = None

    def set_skip(self, arg):
        self.skip = arg

class LinkedList():
    def __init__(self):
        self.head = None
        self.tail = None
        self.current = None
        self.previous = None
        self.current_index = -1
        self.size = 0

    def search(self, arg):
        if self.current != None and self.current.key == arg:
            return True
        self.previous = None
        self.current = self.head
        self.current_index = 0
        while self.current != None:
            if self.current.key == arg:
                return True
            elif self.current.key > arg:
                return False
            else:
                self.previous = self.current
                self.current = self.current.next
                self.current_index += 1
        return False

    def add(self, arg, skip=False):
        if self.search(arg):
            return
        if skip:
            temp = SkipNode(arg)
        else:
            temp = Node(arg)
        if self.previous == None:
            self.head = temp
        else:
            self.previous.set_next(temp)
        temp.set_next(self.current)
        self.size += 1
        if self.current == None:
            self.tail = temp
        self.current = None

    def index(self, item):
        if not self.search(item):
            return -1
        return self.current_index


class SkipList(LinkedList):
    def __init__(self):
        super().__init__()

    def add(self, arg):
        super().add(arg, True)


def intersectWithSkips(lyst1, lyst2):
    answer = LinkedList()
    head1 = lyst1.head
    head2 = lyst2.head

    while head1 and head2:
        if head1.key == head2.key:
            answer.add(head1.key)
            head1 = head1.next
            head2 = head2.next

        elif head1.key < head2.key:
            if head1.skip is not None and head1.skip.key <= head2.key:
                while head1.skip is not None and head1.skip.key <= head2.key:
                    head1 = head1.skip
            else:
                head1 = head1.next
        else:
            if head2.skip is not None and head2.skip.key <= head1.key:
                while head2.skip is not None and head2.skip.key <= head1.key:
                    head2 = head2.skip
            else:
                head2 = head2.next

    return answer

def setPara(lyst, para):
    head = lyst.head
    para = int(para)
    count = 0
    while head:
        if count % para == 0:
            node = skip(head, para)
            if node != None:
                head.set_skip(node)
        count += 1
        head = head.next

def skip(h, p):
    count = 1
    while h and count < p:
        h = h.next
        count += 1
    return h

def print_llist(arg):
    head = arg.head
    print("[ ", end="")
    while head:
        print(head.key, end=", ")
        head = head.next
    print("]")

def s_intersectp(a1, a2):
    answer = SkipList()
    head1 = a1.head
    head2 = a2.head

    while head1 and head2:
        if head1.key == head2.key:
            answer.add(head1.key)
            head1 = head1.next
            head2 = head2.next

        elif head1.key < head2.key:
            if head1.skip is not None and head1.skip.key <= head2.key:
                while head1.skip is not None and head1.skip.key <= head2.key:
                    head1 = head1.skip
            else:
                head1 = head1.next
        else:
            if head2.skip is not None and head2.skip.key <= head1.key:
                while head2.skip is not None and head2.skip.key <= head1.key:
                    head2 = head2.skip
            else:
                head2 = head2.next
    setPara(answer, answer.size)
    return answer

def s_boolq(ql, inv_index):
    ps = PorterStemmer()
    query_list = [ps.stem(w.strip().lower()) for w in ql]
    for i in query_list:
        if i not in inv_index:
            return []
    lens = [(inv_index[query_list[i]].size, inv_index[query_list[i]]) for i in range(len(query_list))]
    lens.sort(key=lambda x: x[0])
    while len(lens) > 1:
        c = s_intersectp(lens.pop()[1], lens.pop()[1])
        bisect.insort(lens, (c.size, c))
    return lens[0][1]



inv_index = {}
stop_words = set(stopwords.words('english')) | {' '}
ps = PorterStemmer()
for p in range(3):
    for q in range(20):
        with open(os.path.join('docs', 'd'+str(q+1)+'_'+'q'+str(p+1)), 'r') as f:
            # split words, strip spaces, change to lower-case, remove punctuation marks, and remove stop-words
            cdoc = [w.strip().lower().translate(str.maketrans('', '', string.punctuation)) for s in f.readlines() for w in s.split(" ") if w.strip().lower().translate(str.maketrans('', '', string.punctuation)) not in stop_words]
            # stemming (and remove empty strings)
            cdoc = [ps.stem(w) for w in cdoc if w!='']
            for i in cdoc:
                if i in inv_index:
                    if not inv_index[i].search(p * 20 + q):
                        inv_index[i].add(p*20+q)
                else:
                    inv_index[i] = SkipList()
                    inv_index[i].add(p*20+q)
for k, v in inv_index.items():
    setPara(inv_index[k], inv_index[k].size ** 0.5)


print_llist(s_boolq(['IIT', 'Computer'], inv_index))
print_llist(s_boolq(['Computer', 'Bhilai'], inv_index))
print_llist(s_boolq(['Bhilai', 'Computer', 'IIT'], inv_index))

[ 34, 37, ]
[ ]
[ ]


In [ ]:
print(timeit.timeit("s_boolq(['IIT', 'Computer'], inv_index)", 'from __main__ import s_boolq, inv_index', number=100))
print(timeit.timeit("s_boolq(['Computer', 'Bhilai'], inv_index)", 'from __main__ import s_boolq, inv_index', number=100))
print(timeit.timeit("s_boolq(['Bhilai', 'Computer', 'IIT'], inv_index)", 'from __main__ import s_boolq, inv_index', number=100))

0.011271630999999616
0.005513260000043374
0.008279890000039813


**Comparison of runtimes: **   
Timings for queries with inverse index from part 1:  
0.009093049000057363  
0.005048134999924514  
0.007286062000048332  
  
Timings for queries with inverse index from part 3:   
0.011271630999999616  
0.005513260000043374  
0.008279890000039813  

Timings in part3 are more because of the additional steps done for implementation.

#Question 4

In [ ]:
inv_index = {}
stop_words = set(stopwords.words('english')) | {' '}
ps = PorterStemmer()
for p in range(3):
    for q in range(20):
        with open(os.path.join('docs', 'd'+str(q+1)+'_'+'q'+str(p+1)), 'r') as f:
            # split words, strip spaces, change to lower-case, remove punctuation marks, and remove stop-words
            cdoc = [w.strip().lower().translate(str.maketrans('', '', string.punctuation)) for s in f.readlines() for w in s.split(" ") if w.strip().lower().translate(str.maketrans('', '', string.punctuation)) not in stop_words]
            # stemming (and remove empty strings)
            cdoc = [ps.stem(w) for w in cdoc if w!='']
            for i in cdoc:
                if i in inv_index:
                    if inv_index[i][-1] != p * 20 + q:
                        inv_index[i].append(p*20+q)
                else:
                    inv_index[i] = [p*20+q]
# we are using l(logarithm) for term frequeny, t(idf) for document frequency and  c(cosine) for normalisation for documents
# we are using n(natural) for term frequency, n(no) i.e 1 for document frequency  and n(none) i.e 1 for normalisation for query


from math import log
# funtion which gives tfidf for a given term
def tfidf(term, documentid):
    count = 0
    for i in inv_index.keys(): # searching required Docid id
    # if Docid id present then we add the frequency of term
        if documentid in inv_index[i]:
          count = count + inv_index[term].count(documentid)
    if inv_index[term].count(documentid) != 0: # if the term present in inv_index we calculate TF-iDF according to the scheme else TF-iDF is 0
        tf = 1 + log(inv_index[term].count(documentid),10)
        idf = log(N/len(set(inv_index[term])),10)
        result = tf * idf
    else:
        result = 0
    return result

# dictionary to store TF-iDF of each term and document
tfidfd = {}
for i in range(N):
    for term in inv_index.keys(): 
        if i == 0:
            tfidfd[term] = [tfidf(term, i)]
        else:
            tfidfd[term].append(tfidf(term, i))

df = pd.DataFrame(tfidfd)
df.head()

,editor,review,you’v,submit,determin,whether,revis,articl,comput,scienc,studi,includ,theoret,algorithm,foundat,hardwar,softwar,use,process,inform,disciplin,data,structur,network,design,model,artifici,intellig,draw,mathemat,engin,therefor,incorpor,techniqu,area,queue,theori,probabl,statist,electron,...,god,rama,foodprocess,rice,wheat,cotton,oilse,sawmil,rail,vizianagaram,vishakhapatnam,port,handloom,weav,furnitur,manufactur,machineri,cigarett,aluminium,brass,bellmet,product,print,press,museum,tuberculosi,clinic,sericultur,farm,pandit,ravishankar,shukla,1964,605747,agglom,700113,1010433,1123558,26,th
0,1.30103,1.176091,1.30103,1.30103,1.30103,1.30103,1.30103,1.079181,0.547702,0.522879,0.933053,0.823909,1.477121,1.477121,1.30103,1.477121,1.30103,0.736759,1.30103,0.875061,1.778151,1.079181,1.30103,1.0,1.30103,1.30103,1.079181,1.0,1.778151,1.30103,0.632023,1.778151,1.778151,1.477121,0.875061,1.778151,1.176091,1.778151,1.778151,1.778151,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.547702,0.522879,0.933053,0.823909,0.000000,0.000000,0.00000,0.000000,1.30103,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,1.0,1.30103,0.00000,1.079181,1.0,0.000000,0.00000,0.632023,0.000000,0.000000,0.000000,0.875061,0.000000,1.176091,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.547702,0.522879,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.0,0.00000,0.00000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.736759,0.00000,0.000000,0.000000,0.000000,0.00000,0.0,0.00000,0.00000,0.000000,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.00000,0.000000,0.00000,0.00000,0.00000,0.00000,0.00000,0.000000,0.547702,0.522879,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000,0.0,0.00000,0.00000,1.079181,1.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# normalization
# list which contains sum of squares of document vectors
d = []
for i in range(N):
    c = 0
    for j in tfidfd.keys():
        c += (tfidfd[j][i])*(tfidfd[j][i])
    d.append(c)
for i in range(N):
    for j in tfidfd.keys():
        if d[i]: # if Euclidean distance of a Doc vector is zero then we give zero to result
            tfidfd[j][i] = tfidfd[j][i]/(d[i]*0.5)
        else:
            tfidfd[j][i] = 0
dw = pd.DataFrame(tfidfd)
dw.head()

,editor,review,you’v,submit,determin,whether,revis,articl,comput,scienc,studi,includ,theoret,algorithm,foundat,hardwar,softwar,use,process,inform,disciplin,data,structur,network,design,model,artifici,intellig,draw,mathemat,engin,therefor,incorpor,techniqu,area,queue,theori,probabl,statist,electron,...,god,rama,foodprocess,rice,wheat,cotton,oilse,sawmil,rail,vizianagaram,vishakhapatnam,port,handloom,weav,furnitur,manufactur,machineri,cigarett,aluminium,brass,bellmet,product,print,press,museum,tuberculosi,clinic,sericultur,farm,pandit,ravishankar,shukla,1964,605747,agglom,700113,1010433,1123558,26,th
0,0.012574,0.011366,0.012574,0.012574,0.012574,0.012574,0.012574,0.01043,0.005293,0.005053,0.009017,0.007963,0.014275,0.014275,0.012574,0.014275,0.012574,0.007120,0.012574,0.008457,0.017185,0.01043,0.012574,0.009664,0.012574,0.012574,0.010430,0.009664,0.017185,0.012574,0.006108,0.017185,0.017185,0.014275,0.008457,0.017185,0.011366,0.017185,0.017185,0.017185,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.021515,0.020540,0.036653,0.032365,0.000000,0.000000,0.000000,0.000000,0.051108,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.039282,0.051108,0.000000,0.042393,0.039282,0.000000,0.000000,0.024827,0.000000,0.000000,0.000000,0.034374,0.000000,0.046200,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.011454,0.010935,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018079,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.039037,0.037268,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.076918,0.071275,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from nltk.stem import PorterStemmer


def scoredoc(ql):
    ps = PorterStemmer()
    q = [ps.stem(i.strip().lower()) for i in ql]
    for i in q:
        if i not in inv_index: # if the given query term is not present in inv_index we return empty list
            return[]
    s = []
    for j in range(N):
        flag = 1
        c = 0
        for i in q:
            if tfidfd[i][j] == 0: # if TF-iDF of a term and document is Zero we check it
                flag = 0
            c += tfidfd[i][j]
        if flag: # TF-iDF not equal 0
            s.append(c)
        else: # TF-iDF equal 0
            s.append(0)
    y = s[:] # copying to another list
    s.sort() # sorting according to score
    rank = []
    for i in range(len(s)):
        rank.append('rank '+str(len(s)-i)+ ' docid'+str(y.index(s[i]))+' score = '+ str(s[i]))
        y[y.index(s[i])] = -1
    rank.reverse()
    return rank[:len(set(s))-1] #returning only document with non zero TF-iDF

In [ ]:
queries = [['computer', 'IIT'], ['IIT', 'Bhilai'], ['computer', 'IIT', 'Bhilai'], ['computer', 'Science']]
for i in queries:
    print(scoredoc(i))

['rank 1 docid37 score = 0.010093189926718385', 'rank 2 docid34 score = 0.009271069667088282']
[]
[]
['rank 1 docid15 score = 0.127992471444988', 'rank 2 docid4 score = 0.0763051359191613', 'rank 3 docid11 score = 0.06777697854556053', 'rank 4 docid1 score = 0.04205496347745674', 'rank 5 docid18 score = 0.02885744226091546', 'rank 6 docid14 score = 0.027146589363792523', 'rank 7 docid6 score = 0.025009522086166677', 'rank 8 docid7 score = 0.02325481096261034', 'rank 9 docid2 score = 0.022389838580960664', 'rank 10 docid10 score = 0.018650702055143464', 'rank 11 docid16 score = 0.015302834540924844', 'rank 12 docid17 score = 0.015256778646743895', 'rank 13 docid0 score = 0.01034648911549654', 'rank 14 docid37 score = 0.008149903038083082', 'rank 15 docid34 score = 0.007486069260033146']
